In [0]:
# ——————————————————————————————
#  LIBRARIES
# ——————————————————————————————    
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# ——————————————————————————————
#  ENVIRONMENT CONFIGURATION
# ——————————————————————————————
dbutils.widgets.text("pipeline.env", "dev") # Create widget with default value - tasks need to have widgets
env = dbutils.widgets.get("pipeline.env")

catalog = "book_rec_catalog"
bronze_schema = f"{env}_bronze"
silver_schema = f"{env}_silver"
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {silver_schema}")

In [0]:
# ————————————————————————————————————————————————
# FINAL NON-STREAMING TABULKY PRO STREAMING
# ————————————————————————————————————————————————

def promote_to_batch_table(table_name: str):
    """
    The function is used to promote the silver tables to classic delta tables without DLT.
    It is used to use column masking and row filtering.
    """
    source_path = f"{catalog}.{silver_schema}.{table_name}"
    target_path = f"{catalog}.{silver_schema}.{table_name}_batch"
    
    print(f"Promoting {source_path} to {target_path}")
    
    try:
        (
            spark.read.table(source_path)
                 .write
                 .mode('overwrite')
                 .option('overwriteSchema', 'true')
                 .saveAsTable(target_path)
        )
        print(f"Successfully promoted {table_name} to {table_name}_batch")
    except Exception as e:
        print(f"Error promoting {table_name}: {str(e)}")

# list of tables to promote
tables_to_promote = [
    'books_silver',
    'ratings_silver', 
    'users_silver'
]

print(f"Starting promotion of {len(tables_to_promote)} tables...")
print("=" * 60)

for table in tables_to_promote:
    promote_to_batch_table(table)

print("=" * 60)
print("Promotion completed!")